## imports

In [22]:
import polars as pl
from polars import col as c
import os, sys, requests
from dotenv import load_dotenv, find_dotenv
from tqdm import tqdm

load_dotenv(find_dotenv())

True

## constants

In [23]:
OMDB_API_KEY = os.getenv("OMDB_API_KEY")
FILES_DIR = "../data"

## load df

In [24]:
file_name = "movies_df.xlsx"
file_path = os.path.join(FILES_DIR, file_name)
movies_df = pl.read_excel(file_path)

Could not determine dtype for column 2, falling back to string


Could not determine dtype for column 3, falling back to string


## code

In [25]:
movies_df.head()

index,Title,Year,Viewed,title_eng,title_translation,parsed_date,parsed_title,checked,gpt_parsed_titles,years,titles,cleaned_title,cleaned_year,ids
i64,str,str,str,str,str,i64,str,str,str,str,str,str,i64,str
1,"""Действуй сестра""",null,null,"""Not found""","""Act sister""",null,"""Not found""",null,"""Sister Act""","""1992""","""Sister Act""","""Sister Act""",1992,"""tt0105417"""
2,"""Summer lovers""",null,null,"""Summer Lovers_1982_Comedy, Dra…","""Summer lovers""",1982,"""Summer Lovers""","""checked""","""Summer lovers""",null,null,"""Summer Lovers""",1982,"""tt0084737"""
3,"""Джон уик""",null,null,"""John Wick_2014_Action, Crime, …","""John Wick""",2014,"""John Wick""","""checked""","""Джон уик""",null,null,"""John Wick""",2014,"""tt2911666"""
4,"""Помни""",null,null,"""Remembered_2015_Short, Drama_N…","""Remembered""",2015,"""Remembered""","""checked""","""Помни""",null,null,"""Memento""",2000,"""tt0209144"""
5,"""Похитители велосипедов""",null,null,"""Not found""","""Cycles of bicycles""",null,"""Not found""",null,"""Bicycle Thieves""","""1948""","""Bicycle Thieves""","""Bicycle Thieves""",1948,"""tt0040522"""


In [ ]:
movies_df.filter(c("ids") == "Not found")

index,Title,Year,Viewed,title_eng,title_translation,parsed_date,parsed_title,checked,gpt_parsed_titles,years,titles,cleaned_title,cleaned_year,ids
i64,str,str,str,str,str,i64,str,str,str,str,str,str,i64,str
193,"""True detective *""",null,null,null,null,null,null,"""checked""","""True detective *""",null,null,null,null,"""Not found"""
355,"""(100 любимых фильмов фрэнка оу…",null,null,"""Not found""","""(100 favorite films Frank Ocea…",null,"""Not found""",null,"""(100 Favorite Films of Frank O…","""-1""",null,null,null,"""Not found"""
450,"""Отбросы *""",null,null,"""Not found""","""Dispers *""",null,"""Not found""",null,"""Misfits""","""2009–2013""","""Misfits""","""Misfits""",null,"""Not found"""
493,"""Мanhattan shorts *""","""2020""",null,"""Not found""","""Мanhattan shorts *""",null,"""Not found""",null,"""Manhattan Shorts""","""-1""",null,null,null,"""Not found"""


In [27]:
def fetch_english_title(title):
    try:
        url = f"http://www.omdbapi.com/?apikey={OMDB_API_KEY}&i={requests.utils.quote(title)}"  # for index search
        # url = f"http://www.omdbapi.com/?apikey={OMDB_API_KEY}&t={requests.utils.quote(title)}"
        response = requests.get(url)
        data = response.json()
        if data.get("Response") == "True" and "Title" in data:
            return data
        else:
            print(f"Not found in OMDb: {title}")
            return "Not found"
    except Exception as e:
        print(f"OMDb error for {title}: {e}")
        return "Not found"

In [28]:
fetch_english_title(movies_df.head(1)["ids"].item())

{'Title': 'Sister Act',
 'Year': '1992',
 'Rated': 'PG',
 'Released': '29 May 1992',
 'Runtime': '100 min',
 'Genre': 'Comedy, Family, Music',
 'Director': 'Emile Ardolino',
 'Writer': 'Paul Rudnick',
 'Actors': 'Whoopi Goldberg, Maggie Smith, Harvey Keitel',
 'Plot': 'When a nightclub singer is forced to take refuge from the mob in a convent, she ends up turning the convent choir into a soulful chorus complete with a Motown repertoire, until the sudden celebrity of the choir jeopardizes her ide...',
 'Language': 'English',
 'Country': 'United States',
 'Awards': '9 wins & 10 nominations total',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMjIzOTU0ODItMzcyMi00YzFjLTk3MzktNTQ2YjZmN2E0NTBkXkEyXkFqcGc@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.5/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '73%'},
  {'Source': 'Metacritic', 'Value': '51/100'}],
 'Metascore': '51',
 'imdbRating': '6.5',
 'imdbVotes': '113,562',
 'imdbID': 'tt0105417',
 'Type': 

In [48]:
result_dict = dict()
for row in tqdm(movies_df.head(10).iter_rows(named=True)):
    if row["ids"] == "Not found":
        result_dict[row["index"]] = "Not found"
    else:
        omdb_result = fetch_english_title(row["ids"])
        if omdb_result != "Not found":
            result_dict[row["index"]] = omdb_result
        else:
            result_dict[row["index"]] = "Not found"

366it [01:37,  4.68it/s]

Not found in OMDb: tt0073580


367it [01:37,  4.69it/s]

Not found in OMDb: tt0257360
Not found in OMDb: tt0064855


369it [01:38,  4.48it/s]

...

Not found in OMDb: tt0093779
Not found in OMDb: tt0095327


620it [02:33,  4.75it/s]

Not found in OMDb: tt0114814


621it [02:33,  4.04it/s]

Not found in OMDb: tt0116905


In [49]:
for i, (key, val) in enumerate(result_dict.items()):
    print(f"{key}\t{val}")
    if i > 9:
        break

1	{'Title': 'Sister Act', 'Year': '1992', 'Rated': 'PG', 'Released': '29 May 1992', 'Runtime': '100 min', 'Genre': 'Comedy, Family, Music', 'Director': 'Emile Ardolino', 'Writer': 'Paul Rudnick', 'Actors': 'Whoopi Goldberg, Maggie Smith, Harvey Keitel', 'Plot': 'When a nightclub singer is forced to take refuge from the mob in a convent, she ends up turning the convent choir into a soulful chorus complete with a Motown repertoire, until the sudden celebrity of the choir jeopardizes her ide...', 'Language': 'English', 'Country': 'United States', 'Awards': '9 wins & 10 nominations total', 'Poster': 'https://m.media-amazon.com/images/M/MV5BMjIzOTU0ODItMzcyMi00YzFjLTk3MzktNTQ2YjZmN2E0NTBkXkEyXkFqcGc@._V1_SX300.jpg', 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.5/10'}, {'Source': 'Rotten Tomatoes', 'Value': '73%'}, {'Source': 'Metacritic', 'Value': '51/100'}], 'Metascore': '51', 'imdbRating': '6.5', 'imdbVotes': '113,562', 'imdbID': 'tt0105417', 'Type': 'movie', 'DVD': 'N/A'

In [50]:
creds = []
indexes = result_dict.keys()
for key in indexes:
    creds.append(result_dict[key])

In [ ]:
parsed_films = pl.from_dict(
    {
        "index": indexes,
        "creds": creds,
    }
)

parsed_films.sample(5)

index,creds
i64,struct[25]
536,"{null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null}"
167,"{""There Will Be Blood"",""2007"",""R"",""25 Jan 2008"",""158 min"",""Drama"",""Paul Thomas Anderson"",""Paul Thomas Anderson, Upton Sinclair"",""Daniel Day-Lewis, Paul Dano, Ciarán Hinds"",""A story of family, religion, hatred, oil and madness, focusing on a turn-of-the-century prospector in the early days of the business."",""English, American Sign "",""United States"",""Won 2 Oscars. 114 wins & 138 nominations total"",""https://m.media-amazon.com/images/M/MV5BMjAxODQ4MDU5NV5BMl5BanBnXkFtZTcwMDU4MjU1MQ@@._V1_SX300.jpg"",[{""Internet Movie Database"",""8.2/10""}, {""Rotten Tomatoes"",""91%""}, {""Metacritic"",""93/100""}],""93"",""8.2"",""674,063"",""tt0469494"",""movie"",""N/A"",""$40,222,514"",""N/A"",""N/A"",""True""}"
290,"{""Blue Velvet"",""1986"",""R"",""23 Oct 1986"",""120 min"",""Crime, Drama, Mystery"",""David Lynch"",""David Lynch"",""Isabella Rossellini, Kyle MacLachlan, Dennis Hopper"",""The discovery of a severed human ear found in a field leads a young man on an investigation related to a beautiful, mysterious nightclub singer and a group of psychopathic criminals who have kidnapped her child."",""English"",""United States"",""Nominated for 1 Oscar. 18 wins & 18 nominations total"",""https://m.media-amazon.com/images/M/MV5BNzkwZGUyYjQtM2QyOC00MTBjLWE0NGYtMTZmOGM5MjI0Mzg0XkEyXkFqcGc@._V1_SX300.jpg"",[{""Internet Movie Database"",""7.7/10""}, {""Rotten Tomatoes"",""91%""}, {""Metacritic"",""75/100""}],""75"",""7.7"",""233,096"",""tt0090756"",""movie"",""N/A"",""$8,551,228"",""N/A"",""N/A"",""True""}"
104,"{""Men in Black"",""1997"",""PG-13"",""02 Jul 1997"",""98 min"",""Action, Adventure, Comedy"",""Barry Sonnenfeld"",""Lowell Cunningham, Ed Solomon"",""Tommy Lee Jones, Will Smith, Linda Fiorentino"",""James, an NYC cop, is hired by Agent K of a secret government agency that monitors extraterrestrial life on Earth. Together, they must recover an item that has been stolen by an intergalactic villain."",""English, Spanish"",""United States"",""Won 1 Oscar. 22 wins & 39 nominations total"",""https://m.media-amazon.com/images/M/MV5BZmQ0YTdhYzEtMjIxZC00MGFiLTlmNjktZmFlOWE2M2QxOTRhXkEyXkFqcGc@._V1_SX300.jpg"",[{""Internet Movie Database"",""7.3/10""}, {""Rotten Tomatoes"",""91%""}, {""Metacritic"",""72/100""}],""72"",""7.3"",""644,209"",""tt0119654"",""movie"",""N/A"",""$250,690,539"",""N/A"",""N/A"",""True""}"
295,"{""Vertigo"",""1958"",""PG"",""28 May 1958"",""128 min"",""Mystery, Romance, Thriller"",""Alfred Hitchcock"",""Alec Coppel, Samuel A. Taylor, Pierre Boileau"",""James Stewart, Kim Novak, Barbara Bel Geddes"",""A former San Francisco police detective juggles wrestling with his personal demons and becoming obsessed with the hauntingly beautiful woman he has been hired to trail, who may be deeply disturbed."",""English"",""United States"",""Nominated for 2 Oscars. 9 wins & 8 nominations total"",""https://m.media-amazon.com/images/M/MV5BNzEyZTUxMWItMGVhYy00Y2NlLTljYTMtYzBmMzI1YmUyMDBkXkEyXkFqcGc@._V1_SX300.jpg"",[{""Internet Movie Database"",""8.2/10""}, {""Rotten Tomatoes"",""93%""}, {""Metacritic"",""100/100""}],""100"",""8.2"",""448,438"",""tt0052357"",""movie"",""N/A"",""$7,863,310"",""N/A"",""N/A"",""True""}"


In [34]:
movies_df.head()

index,Title,Year,Viewed,title_eng,title_translation,parsed_date,parsed_title,checked,gpt_parsed_titles,years,titles,cleaned_title,cleaned_year,ids
i64,str,str,str,str,str,i64,str,str,str,str,str,str,i64,str
1,"""Действуй сестра""",null,null,"""Not found""","""Act sister""",null,"""Not found""",null,"""Sister Act""","""1992""","""Sister Act""","""Sister Act""",1992,"""tt0105417"""
2,"""Summer lovers""",null,null,"""Summer Lovers_1982_Comedy, Dra…","""Summer lovers""",1982,"""Summer Lovers""","""checked""","""Summer lovers""",null,null,"""Summer Lovers""",1982,"""tt0084737"""
3,"""Джон уик""",null,null,"""John Wick_2014_Action, Crime, …","""John Wick""",2014,"""John Wick""","""checked""","""Джон уик""",null,null,"""John Wick""",2014,"""tt2911666"""
4,"""Помни""",null,null,"""Remembered_2015_Short, Drama_N…","""Remembered""",2015,"""Remembered""","""checked""","""Помни""",null,null,"""Memento""",2000,"""tt0209144"""
5,"""Похитители велосипедов""",null,null,"""Not found""","""Cycles of bicycles""",null,"""Not found""",null,"""Bicycle Thieves""","""1948""","""Bicycle Thieves""","""Bicycle Thieves""",1948,"""tt0040522"""


In [53]:
movies_df = movies_df.join(parsed_films, on="index", how="left")

In [54]:
movies_df.head()

index,Title,Year,Viewed,title_eng,title_translation,parsed_date,parsed_title,checked,gpt_parsed_titles,years,titles,cleaned_title,cleaned_year,ids,creds,creds_right
i64,str,str,str,str,str,i64,str,str,str,str,str,str,i64,str,struct[25],struct[25]
1,"""Действуй сестра""",null,null,"""Not found""","""Act sister""",null,"""Not found""",null,"""Sister Act""","""1992""","""Sister Act""","""Sister Act""",1992,"""tt0105417""","{""Sister Act"",""1992"",""PG"",""29 May 1992"",""100 min"",""Comedy, Family, Music"",""Emile Ardolino"",""Paul Rudnick"",""Whoopi Goldberg, Maggie Smith, Harvey Keitel"",""When a nightclub singer is forced to take refuge from the mob in a convent, she ends up turning the convent choir into a soulful chorus complete with a Motown repertoire, until the sudden celebrity of the choir jeopardizes her ide..."",""English"",""United States"",""9 wins & 10 nominations total"",""https://m.media-amazon.com/images/M/MV5BMjIzOTU0ODItMzcyMi00YzFjLTk3MzktNTQ2YjZmN2E0NTBkXkEyXkFqcGc@._V1_SX300.jpg"",[{""Internet Movie Database"",""6.5/10""}, {""Rotten Tomatoes"",""73%""}, {""Metacritic"",""51/100""}],""51"",""6.5"",""113,562"",""tt0105417"",""movie"",""N/A"",""$139,605,150"",""N/A"",""N/A"",""True""}","{""Sister Act"",""1992"",""PG"",""29 May 1992"",""100 min"",""Comedy, Family, Music"",""Emile Ardolino"",""Paul Rudnick"",""Whoopi Goldberg, Maggie Smith, Harvey Keitel"",""When a nightclub singer is forced to take refuge from the mob in a convent, she ends up turning the convent choir into a soulful chorus complete with a Motown repertoire, until the sudden celebrity of the choir jeopardizes her ide..."",""English"",""United States"",""9 wins & 10 nominations total"",""https://m.media-amazon.com/images/M/MV5BMjIzOTU0ODItMzcyMi00YzFjLTk3MzktNTQ2YjZmN2E0NTBkXkEyXkFqcGc@._V1_SX300.jpg"",[{""Internet Movie Database"",""6.5/10""}, {""Rotten Tomatoes"",""73%""}, {""Metacritic"",""51/100""}],""51"",""6.5"",""113,562"",""tt0105417"",""movie"",""N/A"",""$139,605,150"",""N/A"",""N/A"",""True""}"
2,"""Summer lovers""",null,null,"""Summer Lovers_1982_Comedy, Dra…","""Summer lovers""",1982,"""Summer Lovers""","""checked""","""Summer lovers""",null,null,"""Summer Lovers""",1982,"""tt0084737""","{""Summer Lovers"",""1982"",""R"",""16 Jul 1982"",""98 min"",""Comedy, Drama, Romance"",""Randal Kleiser"",""Randal Kleiser"",""Peter Gallagher, Daryl Hannah, Valérie Quennessen"",""A young American couple and a French woman engage in a threesome in the Greek Islands."",""English"",""United States"",""N/A"",""https://m.media-amazon.com/images/M/MV5BMTYxNjk0Mjg1NF5BMl5BanBnXkFtZTcwNzAyMTM3NA@@._V1_SX300.jpg"",[{""Internet Movie Database"",""5.8/10""}, {""Rotten Tomatoes"",""25%""}],""N/A"",""5.8"",""4,687"",""tt0084737"",""movie"",""N/A"",""$4,968,000"",""N/A"",""N/A"",""True""}","{""Summer Lovers"",""1982"",""R"",""16 Jul 1982"",""98 min"",""Comedy, Drama, Romance"",""Randal Kleiser"",""Randal Kleiser"",""Peter Gallagher, Daryl Hannah, Valérie Quennessen"",""A young American couple and a French woman engage in a threesome in the Greek Islands."",""English"",""United States"",""N/A"",""https://m.media-amazon.com/images/M/MV5BMTYxNjk0Mjg1NF5BMl5BanBnXkFtZTcwNzAyMTM3NA@@._V1_SX300.jpg"",[{""Internet Movie Database"",""5.8/10""}, {""Rotten Tomatoes"",""25%""}],""N/A"",""5.8"",""4,687"",""tt0084737"",""movie"",""N/A"",""$4,968,000"",""N/A"",""N/A"",""True""}"
3,"""Джон уик""",null,null,"""John Wick_2014_Action, Crime, …","""John Wick""",2014,"""John Wick""","""checked""","""Джон уик""",null,null,"""John Wick""",2014,"""tt2911666""","{""John Wick"",""2014"",""R"",""24 Oct 2014"",""101 min"",""Action, Crime, Thriller"",""Chad Stahelski"",""Derek Kolstad"",""Keanu Reeves, Michael Nyqvist, Alfie Allen"",""John Wick is a former hitman grieving the loss of his true love. When his home is broken into, robbed, and his dog killed, he is forced to return to action to exact revenge."",""English, Russian, Hungarian"",""United States"",""5 wins & 10 nominations total"

In [37]:
movies_df.head(1)["creds"].item()

{'Title': 'Sister Act',
 'Year': '1992',
 'Rated': 'PG',
 'Released': '29 May 1992',
 'Runtime': '100 min',
 'Genre': 'Comedy, Family, Music',
 'Director': 'Emile Ardolino',
 'Writer': 'Paul Rudnick',
 'Actors': 'Whoopi Goldberg, Maggie Smith, Harvey Keitel',
 'Plot': 'When a nightclub singer is forced to take refuge from the mob in a convent, she ends up turning the convent choir into a soulful chorus complete with a Motown repertoire, until the sudden celebrity of the choir jeopardizes her ide...',
 'Language': 'English',
 'Country': 'United States',
 'Awards': '9 wins & 10 nominations total',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMjIzOTU0ODItMzcyMi00YzFjLTk3MzktNTQ2YjZmN2E0NTBkXkEyXkFqcGc@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.5/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '73%'},
  {'Source': 'Metacritic', 'Value': '51/100'}],
 'Metascore': '51',
 'imdbRating': '6.5',
 'imdbVotes': '113,562',
 'imdbID': 'tt0105417',
 'Type': 

In [66]:
movies_df.with_columns(
    c("creds_right").struct.field("Year").cast(pl.Int64).alias("year_creds"),
    c("creds_right").struct.field("Title").alias("title_creds"),
).filter(c("year_creds") != c("cleaned_year"))

index,Title,Year,Viewed,title_eng,title_translation,parsed_date,parsed_title,checked,gpt_parsed_titles,years,titles,cleaned_title,cleaned_year,ids,creds,creds_right,year_creds,title_creds
i64,str,str,str,str,str,i64,str,str,str,str,str,str,i64,str,struct[25],struct[25],i64,str
181,"""Сыщик *""",null,null,null,null,2007,"""Sleuth""","""checked""","""Сыщик *""",null,null,"""Sleuth""",2007,"""tt0069281""",null,"{""Sleuth"",""1972"",""PG"",""11 Dec 1972"",""138 min"",""Mystery, Thriller"",""Joseph L. Mankiewicz"",""Anthony Shaffer"",""Laurence Olivier, Michael Caine, Alec Cawthorne"",""A man who loves games and theater invites his wife's lover to meet him, setting up a battle of wits with potentially deadly results."",""English, Italian"",""United Kingdom, United States"",""Nominated for 4 Oscars. 5 wins & 11 nominations total"",""https://m.media-amazon.com/images/M/MV5BMDNjNzZiYWItOGY4OC00NjEwLWIxZjEtYzNiOTlmMWNkNWFkXkEyXkFqcGc@._V1_SX300.jpg"",[{""Internet Movie Database"",""7.9/10""}, {""Rotten Tomatoes"",""89%""}],""N/A"",""7.9"",""50,869"",""tt0069281"",""movie"",""N/A"",""$4,081,254"",""N/A"",""N/A"",""True""}",1972,"""Sleuth"""
362,"""Сабрина *""",null,null,null,null,1995,"""Sabrina""","""checked""","""Сабрина *""",null,null,"""Sabrina""",1995,"""tt0047437""",null,"{""Sabrina"",""1954"",""Approved"",""15 Oct 1954"",""113 min"",""Comedy, Drama, Romance"",""Billy Wilder"",""Billy Wilder, Samuel A. Taylor, Ernest Lehman"",""Humphrey Bogart, Audrey Hepburn, William Holden"",""After her return from school in Paris, a playboy finally takes notice of his family's chauffeur's daughter, who's long had a crush on him, but he questions his more serious brother's motives when he warns against getting involved ..."",""English, French"",""United States"",""Won 1 Oscar. 7 wins & 8 nominations total"",""https://m.media-amazon.com/images/M/MV5BNjQ5MWI5YWYtYzYxNi00ODM4LWI4NzEtNzUzYmVkNzRiYzNlXkEyXkFqcGc@._V1_SX300.jpg"",[{""Internet Movie Database"",""7.6/10""}, {""Rotten Tomatoes"",""89%""}, {""Metacritic"",""72/100""}],""72"",""7.6"",""73,444"",""tt0047437"",""movie"",""N/A"",""N/A"",""N/A"",""N/A"",""True""}",1954,"""Sabrina"""


## save df

In [16]:
file_name = "movies_df.xlsx"
file_path = os.path.join(FILES_DIR, file_name)
movies_df.write_excel(file_path)